In [1]:
# !! pip install arff

['Collecting arff',
 '  Downloading https://files.pythonhosted.org/packages/50/de/62d4446c5a6e459052c2f2d9490c370ddb6abc0766547b4cef585913598d/arff-0.9.tar.gz',
 'Building wheels for collected packages: arff',
 '  Building wheel for arff (setup.py) ... \x1b[?25l\x1b[?25hdone',
 '  Stored in directory: /root/.cache/pip/wheels/04/d0/70/2c73afedd3ac25c6085b528742c69b9587cbdfa67e5194583b',
 'Successfully built arff',
 'Installing collected packages: arff',
 'Successfully installed arff-0.9']

In [0]:
import arff
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from operator import attrgetter, itemgetter
from io import StringIO
from sklearn.model_selection import train_test_split
import nltk
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, confusion_matrix,classification_report,precision_recall_fscore_support as score, average_precision_score
from sklearn import metrics
from sklearn.model_selection import cross_validate
from pickle import Pickler, Unpickler

## training model
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [3]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [21]:
root_path = "gdrive/My Drive/Colab Notebooks/"
df = Unpickler(open(root_path + "dataset/OffComBR3.sav", 'rb')).load()

X = df['sentence'].tolist()
y = df['hate'].tolist()


nltk.download('stopwords')
nltk.download('rslp')
stopwords = nltk.corpus.stopwords.words('portuguese')
stemmer = nltk.stem.RSLPStemmer()
nltk.download('punkt')

def clean_text(txt):
    text = ''
    for w in nltk.word_tokenize(txt):
        if w not in stopwords:
            text = text + stemmer.stem(w) + ' '
    return text.strip()


for i in range(len(X)):
    n_txt = clean_text(X[i])
    X[i] = n_txt

X_train, X_test, y_train, y_test = train_test_split(
                                X, y, test_size=0.33, random_state=42)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
classifiers = [DecisionTreeClassifier(), MLPClassifier(), MultinomialNB(), RandomForestClassifier(), SGDClassifier(), SVC()]
# classifiers = [SVC()]

In [0]:
def get_recal_1(y_true, y_pred):
    precision, recall, fscore, support = score(y_true, y_pred)
#     print(classification_report(y_true, y_pred))
    return(recall[1])

In [0]:
## classificadores com os melhores hiperparametros

dt =  [TfidfVectorizer(max_df = 0.6,
                                 min_df = 1,
                                 ngram_range = (1, 1),
                                 smooth_idf = True,
                                 strip_accents = 'ascii',
                                 sublinear_tf = False,
                                 use_idf = True),
        DecisionTreeClassifier(class_weight= {0: 1, 1: 2}, criterion= 'gini', min_samples_split = 3),
      False, 0.5, 0.2, 2]


mlp =  [TfidfVectorizer(max_df= 0.2,
                                 min_df= 5,
                                 ngram_range= (1, 2),
                                 smooth_idf= True,
                                 strip_accents= 'ascii',
                                 sublinear_tf= False,
                                 use_idf = False),
        MLPClassifier(activation='logistic', alpha= 0, solver='lbfgs'),
       True, 1.0, 1.0, 10]


mnb =  [TfidfVectorizer(max_df= 0.6,
                                 min_df= 5,
                                 ngram_range= (1, 1),
                                 smooth_idf= True,
                                 strip_accents= 'ascii',
                                 sublinear_tf= True,
                                 use_idf = True),
        MultinomialNB(alpha=0.1, fit_prior=False),
       False, 0.2, 1.0, 7]

rf = [TfidfVectorizer(max_df= 0.2,
                                 min_df= 2,
                                 ngram_range= (1, 1),
                                 smooth_idf= False,
                                 strip_accents= 'ascii',
                                 sublinear_tf= True,
                                 use_idf = True),
        RandomForestClassifier(max_depth=None, min_samples_leaf= 1, min_samples_split= 2, n_estimators=100),
     False, 0.9, 1.0, 10]


sgd = [TfidfVectorizer(max_df= 0.2,
                                 min_df= 10,
                                 ngram_range= (1, 3),
                                 smooth_idf= True,
                                 strip_accents= 'ascii',
                                 sublinear_tf= True,
                                 use_idf = False),
       SGDClassifier(alpha=0.01, loss='perceptron', penalty='none'),
      True, 1.0, 0.9, 7]


svc =  [TfidfVectorizer(max_df= 0.6,
                                 min_df= 1,
                                 ngram_range= (1, 1),
                                 smooth_idf= True,
                                 strip_accents= 'ascii',
                                 sublinear_tf= False,
                                 use_idf = True),
        SVC(C=4, kernel ='linear', probability=True, shrinking=True, tol=1),
       False, 1.0, 1.0, 4]

# improved_classifiers = [('dt', dt), ('mlp', mlp), ('mnb', mnb), ('rf', rf), ('sgd', sgd), ('svc', svc)]
improved_classifiers = [dt, mlp, mnb, rf, sgd, svc]

In [48]:
## Otimizando hiperparâmetros do Bagging

for classifier in improved_classifiers:

  bagging = Pipeline([
          ('tfidf', classifier[0]),
          ('bag', BaggingClassifier(base_estimator=classifier[1]))
       ])

  bagging_parameters = {
      'bag__n_estimators' : [2, 4, 7, 10],
      'bag__max_samples' : [0.2, 0.5, 0.9, 1.0],
      'bag__max_features' : [0.2, 0.5, 0.9, 1.0],
      'bag__bootstrap' : [True, False]
  }


  gs = GridSearchCV(bagging, bagging_parameters, cv=5, iid=False, n_jobs=-1, scoring='recall')
  gs.fit(X, y)
  print(classifier[1].__class__.__name__)
  print(gs.best_score_)
  print(gs.best_params_)

DecisionTreeClassifier
0.4909756097560975
{'bag__bootstrap': False, 'bag__max_features': 0.5, 'bag__max_samples': 0.2, 'bag__n_estimators': 2}
MLPClassifier
0.4315853658536585
{'bag__bootstrap': True, 'bag__max_features': 1.0, 'bag__max_samples': 1.0, 'bag__n_estimators': 10}
MultinomialNB
0.5698780487804879
{'bag__bootstrap': False, 'bag__max_features': 0.2, 'bag__max_samples': 1.0, 'bag__n_estimators': 7}
RandomForestClassifier
0.3119512195121951
{'bag__bootstrap': False, 'bag__max_features': 0.9, 'bag__max_samples': 1.0, 'bag__n_estimators': 10}
SGDClassifier
0.37121951219512195
{'bag__bootstrap': True, 'bag__max_features': 1.0, 'bag__max_samples': 0.9, 'bag__n_estimators': 7}
SVC
0.31670731707317074
{'bag__bootstrap': False, 'bag__max_features': 1.0, 'bag__max_samples': 1.0, 'bag__n_estimators': 4}


In [50]:
## Testando bagging com parâmetros otimizados

for classifier in improved_classifiers:

  bagging = Pipeline([
          ('tfidf', classifier[0]),
          ('bag', BaggingClassifier(base_estimator=classifier[1],
                                    bootstrap=classifier[2],
                                    max_features=classifier[3],
                                    max_samples=classifier[4],
                                    n_estimators=classifier[5]))
       ])
  print(classifier[1].__class__.__name__)
  
  bagging.fit(X_train, y_train) 
  ## Report normal
  pred = bagging.predict(X_test)
  print(classification_report(y_test, pred))
  
  precision = cross_val_score(bagging, X, y, cv=10, scoring='precision')
  print("Precision: %0.2f (+/- %0.2f) [%s]" % (precision.mean(), precision.std(), classifier[1].__class__.__name__))
  recall = cross_val_score(bagging, X, y, cv=10, scoring='recall')
  print("Recall: %0.2f (+/- %0.2f) [%s]" % (recall.mean(), recall.std(), classifier[1].__class__.__name__))
  accuracy = cross_val_score(bagging, X, y, cv=10, scoring='accuracy')
  print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (accuracy.mean(), accuracy.std(), classifier[1].__class__.__name__))

  print("=================================================================")
  
  


DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.84      0.71      0.77       267
           1       0.33      0.53      0.41        74

    accuracy                           0.67       341
   macro avg       0.59      0.62      0.59       341
weighted avg       0.73      0.67      0.69       341

Precision: 0.28 (+/- 0.13) [DecisionTreeClassifier]
Recall: 0.27 (+/- 0.21) [DecisionTreeClassifier]
Accuracy: 0.74 (+/- 0.10) [DecisionTreeClassifier]
MLPClassifier
              precision    recall  f1-score   support

           0       0.82      0.92      0.87       267
           1       0.49      0.27      0.35        74

    accuracy                           0.78       341
   macro avg       0.65      0.60      0.61       341
weighted avg       0.75      0.78      0.75       341

Precision: 0.36 (+/- 0.11) [MLPClassifier]
Recall: 0.36 (+/- 0.10) [MLPClassifier]
Accuracy: 0.74 (+/- 0.07) [MLPClassifier]
MultinomialNB
              prec